# Scheduler Experiments

This notebook explores SAGA schedulers interactively.

In [ ]:
from saga.schedulers import HeftScheduler, CpopScheduler, MinMinScheduler
from saga.data import TaskGraph, Network, Task, Data, Processor, Link
import matplotlib.pyplot as plt

## 1. Create a Task Graph

In [ ]:
def make_diamond_graph():
    """Classic diamond: T0 -> T1,T2 -> T3"""
    tg = TaskGraph()
    tg.add_task(Task(name="T0", comp_costs={"P0": 14, "P1": 16, "P2": 9}))
    tg.add_task(Task(name="T1", comp_costs={"P0": 13, "P1": 19, "P2": 18}))
    tg.add_task(Task(name="T2", comp_costs={"P0": 11, "P1": 13, "P2": 19}))
    tg.add_task(Task(name="T3", comp_costs={"P0": 13, "P1": 8, "P2": 17}))
    
    tg.add_data(Data(name="d01", size=18), "T0", "T1")
    tg.add_data(Data(name="d02", size=12), "T0", "T2")
    tg.add_data(Data(name="d13", size=9), "T1", "T3")
    tg.add_data(Data(name="d23", size=15), "T2", "T3")
    return tg

task_graph = make_diamond_graph()
print(f"Tasks: {[t.name for t in task_graph.tasks]}")

## 2. Create a Network

In [ ]:
def make_network(n_processors=3):
    """Fully connected network."""
    net = Network()
    procs = [f"P{i}" for i in range(n_processors)]
    
    for p in procs:
        net.add_processor(Processor(name=p, speed=1.0))
    
    for src in procs:
        for dst in procs:
            if src != dst:
                net.add_link(Link(name=f"L_{src}_{dst}", source=src, dest=dst, bandwidth=1.0))
    return net

network = make_network(3)
print(f"Processors: {[p.name for p in network.processors]}")

## 3. Run HEFT

In [ ]:
heft = HeftScheduler()
schedule = heft.schedule(network, task_graph)

print(f"Makespan: {schedule.makespan}")
print("\nAssignments:")
for task, proc in schedule.task_to_processor.items():
    start = schedule.task_to_start_time[task]
    end = schedule.task_to_end_time[task]
    print(f"  {task} -> {proc} [{start:.1f} - {end:.1f}]")

## 4. Compare with CPoP

In [ ]:
cpop = CpopScheduler()
schedule_cpop = cpop.schedule(network, task_graph)

print(f"HEFT makespan: {schedule.makespan}")
print(f"CPoP makespan: {schedule_cpop.makespan}")
print(f"Difference: {abs(schedule.makespan - schedule_cpop.makespan):.2f}")

## 5. Experiment: Vary Communication Costs

What happens when communication is more expensive?

In [ ]:
# TODO: Create graphs with different communication-to-computation ratios
# and see how HEFT vs CPoP performance changes

## 6. Experiment: Vary Number of Processors

Does adding more processors always help?

In [ ]:
results = []
for n_proc in [2, 3, 4, 5, 6]:
    net = make_network(n_proc)
    # Need to update task graph costs for new processors
    # TODO: implement this experiment
    pass